In [103]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import collections

In [104]:
def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

In [105]:
with open('belling_the_cat.txt', 'r') as f:
    train = f.read().split()
    words = list(set(train))
    n_voc = len(words)
    word2idx, idx2word = dict(), dict()
    for i, freq in enumerate(collections.Counter(train).most_common()):
        word = freq[0]
        word2idx[word] = i
        idx2word[i] = word
    
    word2idx, idx2word = build_dataset(train)

In [106]:
tf.reset_default_graph()
class RNN():
    pass
model = RNN()
n_input = 3
n_hidden = 512
model.W = tf.Variable(tf.random_normal((n_hidden, n)))
model.b = tf.Variable(tf.random_normal([n]))
model.x = tf.placeholder(tf.float32, (None, n_input))
model.ref = tf.placeholder(tf.float32, (None, n))
model.x_seq = tf.split(model.x, n_input, 1)
model.cell = rnn.BasicLSTMCell(n_hidden)

outputs, states = rnn.static_rnn(model.cell, model.x_seq, dtype=tf.float32)
model.pred = tf.matmul(outputs[-1], model.W) + model.b
model.cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=model.pred, labels=model.ref))

In [107]:
rate = 0.001
def onehot(n, i):
    x = np.zeros((n), dtype=float)
    x[i] = 1
    return x

optimizer = tf.train.RMSPropOptimizer(rate).minimize(model.cost)
correct = tf.equal(tf.argmax(model.pred, 1), tf.argmax(model.ref))
model.accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
s = tf.Session()
s.run(tf.global_variables_initializer())

In [ ]:
epochs = 50000
display_step = 10000

print("Starting...")
total_acc = 0
for epoch in range(epochs):
    offset = epoch % (len(train) - n_input)
    idx = np.asarray([[word2idx[w] for w in train[offset: offset+n_input]]])
    ref = np.asarray([onehot(n, getidx(offset+n_input))])
    feed = {model.x: idx, model.ref: ref}
    _, acc, cost, pred = s.run([optimizer, model.accuracy, model.cost, model.pred], feed)
    total_acc += acc
    if (epoch+1) % display_step == 0:
        print "kens_debug", acc
        print "[%d] Avg. Acc. = %.2f" % (epoch+1, (total_acc / display_step * 100))
        words_in = train[offset:offset+n_input]
        word_out = train[offset+n_input]
        j = s.run(tf.argmax(pred, 1))
        word_pred = idx2word[j[0]]
        print "%s => %s [%s]" % (words_in, word_out, word_pred)
        total_acc = 0
print("Done")

Starting...
